In [32]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

import chemO_funcs #import o2sat
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

# %cd phyto
%run PLT.py


In [33]:
df = PLT.get_hydrocat('2022-01-01', '2022-12-30', 'PLT')
df

,TmStamp,hydrocatStart,hydrocatTemperature,hydrocatConductivity,hydrocatDissOxygen,hydrocatSalinity,hydrocatFluorescence,hydrocatTurbidity,hydrocatPH,DateTime,pH total
326,2022-01-04T09:31:15.400Z,None,6.5078,31.0263,9.592,30.7871,7.478,30.272,8.04,2022-01-04 04:31:15.400000,7.934479
331,2022-01-04T10:46:15.400Z,None,6.3631,30.9445,9.663,30.8278,1.844,3.473,8.04,2022-01-04 05:46:15.400000,7.934814
332,2022-01-04T11:01:15.200Z,None,6.3063,30.8235,9.633,30.7459,2.373,15.717,8.04,2022-01-04 06:01:15.200000,7.934773
333,2022-01-04T11:16:15.200Z,None,6.2367,30.7264,9.680,30.7015,3.158,5.010,8.04,2022-01-04 06:16:15.200000,7.934821
334,2022-01-04T11:31:15.200Z,None,6.2593,30.6975,9.698,30.6492,2.170,7.818,8.03,2022-01-04 06:31:15.200000,7.924688
...,...,...,...,...,...,...,...,...,...,...,...
31734,2022-12-29T22:46:29.000Z,12/29/2022 22:45:00,3.9276,27.7203,10.546,29.3688,0.097,64.257,8.12,2022-12-29 17:46:29,8.016546
31735,2022-12-29T23:01:29.000Z,12/29/2022 23:00:00,3.9210,27.7232,10.552,29.3782,0.093,64.640,8.11,2022-12-29 18:01:29,8.006574
31736,2022-12-29T23:16:29.000Z,12/29/2022 23:15:00,3.9183,27.7297,10.567,29.3882,0.123,64.546,8.12,2022-12-29 18:16:29,8.016597
31737,2022-12-29T23:31:29.000Z,12/29/2022 23:30:00,3.9342,27.7421,10.652,29.3883,0.123,65.017,8.13,2022-12-29 18:31:29,8.026570
